In [38]:
import os

import numpy as np
import pandas as pd

In [39]:
# Read all pk1 PR files from Data/pr (sub)folders
pickled_dirs = []
repo_names = []
for folder in os.listdir('../Data/pr'):
    try:
        pickled_dirs.append(f'../Data/pr/{folder}/'+os.listdir('../Data/pr/'+folder)[0])
        repo_names.append(folder)
    except:
        print('NO PULL REQUEST DATA IN: ', folder)
        continue

print('\n', pickled_dirs[:20])
print('\n', repo_names[:20])

NO PULL REQUEST DATA IN:  prisma-graphql-import

 ['../Data/pr/AE9RB-ruby-redis/AE9RB_ruby-redis_pr.pk1', '../Data/pr/AigeStudio-Android/AigeStudio_Android_pr.pk1', '../Data/pr/AlburIvan-SlickForm/AlburIvan_SlickForm_pr.pk1', '../Data/pr/AlexLiuSheng-CheckVersionLib/AlexLiuSheng_CheckVersionLib_pr.pk1', '../Data/pr/AllenDowney-ThinkJavaCode/AllenDowney_ThinkJavaCode_pr.pk1', '../Data/pr/Aufree-ESTMusicIndicator/Aufree_ESTMusicIndicator_pr.pk1', '../Data/pr/Azure-azure-storage-fuse/Azure_azure-storage-fuse_pr.pk1', '../Data/pr/BMIRDS-deepslide/BMIRDS_deepslide_pr.pk1', '../Data/pr/BetterCloud-vault-java-driver/BetterCloud_vault-java-driver_pr.pk1', '../Data/pr/BreakingMalwareResearch-eleven/BreakingMalwareResearch_eleven_pr.pk1', '../Data/pr/BurntSushi-quickcheck/BurntSushi_quickcheck_pr.pk1', '../Data/pr/BycorSanchez-resources/BycorSanchez_resources_pr.pk1', '../Data/pr/CMU-Perceptual-Computing-Lab-openpose/CMU-Perceptual-Computing-Lab_openpose_pr.pk1', '../Data/pr/CVCalendar-CVCalenda

In [40]:
# Read in all pickled files
df_list = [pd.read_pickle(file_dir) for file_dir in pickled_dirs]

In [41]:
df_list[6][:5]

,author,authorAssociation,bodyText,closedAt,comments,createdAt,files,mergedBy,resourcePath,state,title,updatedAt
0,"{'login': 'asorrin-msft', 'company': None}",CONTRIBUTOR,,2017-09-14T21:25:56Z,"{'totalCount': 1, 'nodes': [{'author': {'login...",2017-09-14T20:06:27Z,{'totalCount': 98},{'login': 'asorrin-msft'},/Azure/azure-storage-fuse/pull/1,MERGED,Merge to master,2017-09-14T21:25:56Z
1,"{'login': 'JasonYang-MSFT', 'company': 'MSFT'}",MEMBER,Fixed several issues.\nAdded rename feature.\n...,2017-09-21T04:02:13Z,"{'totalCount': 1, 'nodes': [{'author': {'login...",2017-09-18T10:12:22Z,{'totalCount': 100},{'login': 'vinjiang'},/Azure/azure-storage-fuse/pull/2,MERGED,Dev,2017-09-21T04:02:13Z
2,"{'login': 'seguler', 'company': 'Amazon'}",CONTRIBUTOR,,2017-09-26T04:20:30Z,"{'totalCount': 1, 'nodes': [{'author': {'login...",2017-09-22T03:26:08Z,{'totalCount': 5},{'login': 'seguler'},/Azure/azure-storage-fuse/pull/3,MERGED,Convert to GnuTLS & fix mount script,2017-09-26T04:20:30Z
3,"{'login': 'seguler', 'company': 'Amazon'}",CONTRIBUTOR,,2017-10-12T19:59:37Z,"{'totalCount': 2, 'nodes': [{'author': {'login...",2017-09-26T22:31:50Z,{'totalCount': 5},{'login': 'seguler'},/Azure/azure-storage-fuse/pull/4,MERGED,Adding NOTICES file for TinyXML2,2017-10-12T19:59:37Z
4,"{'login': 'asorrin-msft', 'company': None}",CONTRIBUTOR,,2017-10-30T16:23:48Z,"{'totalCount': 0, 'nodes': []}",2017-10-25T04:51:31Z,{'totalCount': 13},{'login': 'asorrin-msft'},/Azure/azure-storage-fuse/pull/5,MERGED,Dev,2017-10-30T16:23:48Z


In [46]:
def clean_pr_df(pr_df: pd.DataFrame) -> pd.DataFrame:
    """Takes a DataFrame of issue data and returns a cleaned DataFrame."""

    author_dict_list = pr_df['author'].values.tolist()
    unpacked_dict_list = []

    for item in author_dict_list:
        # If `item` is empty append dict of NaNs, else append data
        if item == None:
            unpacked_dict_list.append({'login': np.NaN, 'company': np.NaN})
        else:
            unpacked_dict_list.append(item)
    
    # Recombine issue_df and unpacked dicts
    author_pr_df = pd.DataFrame(unpacked_dict_list)
    clean_pr_df = pr_df.drop(labels=['author'], axis='columns')
    clean_pr_df['author'] = author_pr_df['login']
    clean_pr_df['company'] = author_pr_df['company']

    # Unpack `files` column and `mergedBy` column
    files_dict_list = pr_df['files'].values.tolist()
    unpacked_dict_list = []
    
    for item in files_dict_list:
        # If `item` is empty append dict of NaNs, else append data
        if item == None:
            unpacked_dict_list.append({'totalCount': np.NaN})
        else:
            unpacked_dict_list.append(item)
    
    files_pr_df = pd.DataFrame(unpacked_dict_list)
    clean_pr_df = clean_pr_df.drop(labels=['files'], axis='columns')
    clean_pr_df['files'] = files_pr_df
    
    mergedBy_dict_list = pr_df['mergedBy'].values.tolist()
    unpacked_dict_list = []
    
    for item in mergedBy_dict_list:
        # If `item` is empty append dict of NaNs, else append data
        if item == None:
            unpacked_dict_list.append({'login': np.NaN})
        else:
            unpacked_dict_list.append(item)
    
    mergedBy_pr_df = pd.DataFrame(unpacked_dict_list)
    clean_pr_df = clean_pr_df.drop(labels=['mergedBy'], axis='columns')
    clean_pr_df['mergedBy'] = mergedBy_pr_df

    
    # Type conversion
    try:
        clean_pr_df['createdAt'] = clean_pr_df['createdAt'].apply(pd.Timestamp)
        clean_pr_df['updatedAt'] = clean_pr_df['updatedAt'].apply(pd.Timestamp)
        clean_pr_df['closedAt'] = clean_pr_df['closedAt'].apply(pd.Timestamp)
    except:
        pass
    return clean_pr_df

In [48]:
# Clean each issue DataFrame, record indices of 0-length DataFrames (for pickling)
cleaned_PRs = []
empty_df_ind = []
for ind, df in enumerate(df_list):
    try:
        cleaned_PRs.append(clean_pr_df(df))
    except:
        empty_df_ind.append(ind)

In [49]:
len(cleaned_PRs), len(empty_df_ind), len(repo_names)

(673, 126, 799)

In [50]:
cleaned_dir = '../Data/cleaned_PRs/'
for ind, df in enumerate(cleaned_PRs):
    if ind not in empty_df_ind:
        df.to_pickle(cleaned_dir+repo_names[ind]+'_pr.pk1')

In [53]:
# Test pickled files
pd.read_pickle(cleaned_dir+repo_names[3]+'_pr.pk1')[:5]

,authorAssociation,bodyText,closedAt,comments,createdAt,resourcePath,state,title,updatedAt,author,company,files,mergedBy
0,NONE,,2017-09-29 18:34:56+00:00,"{'totalCount': 0, 'nodes': []}",2017-09-29 18:34:10+00:00,/AllenDowney/ThinkJavaCode/pull/2,CLOSED,Review.ap01.ch07 change,2017-09-29 18:35:02+00:00,JameT55,None,1,NaN
1,CONTRIBUTOR,"Let students directly edit, compile and run ja...",2017-11-15 13:09:56+00:00,"{'totalCount': 2, 'nodes': [{'author': {'login...",2017-11-15 11:50:21+00:00,/AllenDowney/ThinkJavaCode/pull/3,MERGED,Add link to source code hosted on Codiva onlin...,2017-11-27 07:14:52+00:00,jayaprabhakar,None,1,AllenDowney
2,NONE,,2018-10-30 01:30:17+00:00,"{'totalCount': 0, 'nodes': []}",2018-10-29 13:03:15+00:00,/AllenDowney/ThinkJavaCode/pull/4,CLOSED,Create supcom,2018-10-30 01:30:17+00:00,LeooLeoo,None,1,NaN
